# Closeness Centrality
_Closeness Centrality_ is a way of detecting nodes that are able to spread information very efficiently through a graph.

The _Closeness Centrality_ of a node measures its average distance to all other nodes.
Nodes with a high closeness score have the shortest distances to all other nodes.

First we'll import the Neo4j driver and Pandas libraries:


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

Next let's create an instance of the Neo4j driver which we'll use to execute our queries.


In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

Now let's create a sample graph that we'll run the algorithm against.


In [3]:
create_graph_query = '''
CREATE (a:Node{id:"A"}),
       (b:Node{id:"B"}),
       (c:Node{id:"C"}),
       (d:Node{id:"D"}),
       (e:Node{id:"E"})
CREATE (a)-[:LINK]->(b),
       (b)-[:LINK]->(a),
       (b)-[:LINK]->(c),
       (c)-[:LINK]->(b),
       (c)-[:LINK]->(d),
       (d)-[:LINK]->(c),
       (d)-[:LINK]->(e),
       (e)-[:LINK]->(d);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 5, 'relationships-created': 8, 'nodes-created': 5, 'properties-set': 5}


Finally we can run the algorithm by executing the following query:


In [4]:
streaming_query = """
CALL algo.closeness.stream('Node', 'LINK')
YIELD nodeId, centrality

MATCH (n:Node) WHERE id(n) = nodeId

RETURN n.id AS node, centrality
ORDER BY centrality DESC
limit 20;
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,node,centrality
0,C,0.666667
1,B,0.571429
2,D,0.571429
3,A,0.400000
4,E,0.400000


"C" is the best connected node in this graph although "B" and "D" aren't far behind.
"A" and "E" don't have close ties to many other nodes so their scores are lower.
A score of 1 would indicate that a node has a direct connection to all other nodes.